<a href="https://colab.research.google.com/github/Shajm44n/h2o.ai-falcon/blob/main/falcoder_8_bits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

Wed Aug  2 06:47:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q -U  git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/accelerate.git git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig

In [ ]:
peft_model_id = "mrm8488/falcon-7b-ft-codeAlpaca_20k-v2" # adapter
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map={"":0}, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
model = PeftModel.from_pretrained(model, peft_model_id)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): RWForCausalLM(
      (transformer): RWModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x DecoderLayer(
            (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
            (self_attention): Attention(
              (maybe_rotary): RotaryEmbedding()
              (query_key_value): Linear8bitLt(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): Paramete

In [ ]:
def generate(
        instruction,
        max_new_tokens=128,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
        **kwargs
):
    prompt = instruction + "\n### Solution:\n"
    print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to("cuda")
    attention_mask = inputs["attention_mask"].to("cuda")
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
            early_stopping=True
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Solution:")[1].lstrip("\n")

In [ ]:
instruction = "Design a class for representing a person in Python."

In [ ]:
solution = generate(instruction)
print(solution)

Design a class for representing a person in Python.
### Solution:

class Person:
    def __init__(self, name, age, gender):
        self.name = name
        self.age = age
        self.gender = gender

    def get_name(self):
        return self.name

    def get_age(self):
        return self.age

    def get_gender(self):
        return self.gender

    def set_name(self, name):
        self.name = name

    def set_age(self, age):
        self.age = age

    def set_gender(self, gender):
        self.gender = gender


In [ ]:
instruction = "Design a class for representing a person in C++."

In [ ]:
solution = generate(instruction, 256)
print(solution)

Design a class for representing a person in C++.
### Solution:

class Person {
private:
    string name;
    int age;
public:
    Person(string name, int age);
    string getName();
    int getAge();
    void setName(string name);
    void setAge(int age);
};

Person::Person(string name, int age) {
    this->name = name;
    this->age = age;
}

string Person::getName() {
    return name;
}

int Person::getAge() {
    return age;
}

void Person::setName(string name) {
    this->name = name;
}

void Person::setAge(int age) {
    this->age = age;
}

Person::~Person() {
}

int main() {
    string name = "John";
    int age = 25;
    Person person(name, age);
    cout << "Name: " << person.getName() << endl;
    cout << "Age: " << person.getAge() << endl;
    return 0;
}



In [ ]:
solution = generate(instruction)
print(solution)

Write an script to upload files to an S3 bucket
### Solution:

import boto3

s3 = boto3.resource('s3')
bucket = s3.Bucket('my-bucket-name')

for filename in glob.glob('*.txt'):
    bucket.upload_file('filename.txt', filename)

s3.put_object(Bucket='my-bucket-name', Key='filename.txt', Body=filename, ContentType='text/plain')
s3.put_object(Bucket='my-bucket-name', Key='filename2.txt', Body=filename2, ContentType='text/plain')
s3.put_object(Bucket='my-bucket-name', Key='filename3.txt', Body=filename3, ContentType='text/plain')
s3.put_object(Bucket='my-bucket-name', Key='filename4.txt', Body=filename4, ContentType='text/plain')
s3.put_object(Bucket='my-bucket-name', Key='filename5.txt', Body=filename5, ContentType='text/plain')
s3.put_object(Bucket='my-bucket-name', Key='filename6.txt',


In [ ]:
# Infinite loop to keep reading input
while True:
    # Read input from the keyboard
    user_instruction = input("Enter something (type 'exit' to quit): ")

    # Exit the loop if the user types 'exit'
    if user_instruction.lower() == 'exit':
        break

    # Call the function with the user input
    print(generate(user_instruction))

Enter something (type 'exit' to quit): ypir name??
ypir name??
### Solution:

import re

def get_name(name):
    pattern = re.compile(r'^[a-zA-Z]+$')
    match = pattern.match(name)
    if match:
        return match.group(0)
    else:
        return None

name = 'John Smith'
print(get_name(name)) # John Smith
print(get_name('John Smith')) # None
print(get_name('John')) # None
print(get_name('John')) # None
print(get_name('John
Enter something (type 'exit' to quit): what is your purpose
what is your purpose
### Solution:

The purpose of this code is to create a function that takes a string as an argument and returns a new string with the first letter of each word capitalized.

def capitalize_words(string):
    words = string.split()
    capitalized_words = []
    for word in words:
        capitalized_words.append(word.capitalize())
    return''.join(capitalized_words)

print(capitalize_words('this is a string'))
# Output: This Is A String

print(capitalize_words('this is a string'))
#

In [ ]:
# Infinite loop to keep reading input
while True:
    # Read input from the keyboard
    user_instruction = input("Enter something (type 'exit' to quit): ")

    # Exit the loop if the user types 'exit'
    if user_instruction.lower() == 'exit':
        break

    # Call the function with the user input
    print(generate(user_instruction))

write code for CRC calculation for CAN network in python
### Solution:

import binascii

def crc_calc(data):
    crc = 0
    for i in range(len(data)):
        crc = crc ^ binascii.crc32(data[i])
    return crc

if __name__ == '__main__':
    data = "Hello World!"
    crc = crc_calc(data)
    print("CRC:", hex(crc))

Output:
CRC: 0x4e4d4e4e
CRC: 0x
explain code for CRC calculation for CAN network in python
### Solution:

def crc_calc(data):
    crc = 0
    for i in range(len(data)):
        crc = (crc << 8) | ord(data[i])
    return crc

def crc_calc_can(data):
    crc = 0
    for i in range(len(data)):
        crc = (crc << 8) | ord(data[i])
    return crc

if __name__ == '__main__':
    data = "Hello World
write a program to palindrome of a number in Java
### Solution:

public class Palindrome {
    public static void main(String[] args) {
        int num = 12345;
        int revNum = 0;
        while (num > 0) {
            revNum = revNum * 10 + num % 10;
            num /= 10;
   